# Introduction to yabadaba: Save settings across sessions

The yabadaba package also provides a means for users to save certain settings and options that can be loaded again later in a separate Python session.  This Notebook describes how to change those settings for basic users, and how more advanced users can extend the capability to include additional project-specific settings.

Outline of this Notebook:

1. Saving and loading database host and authentication information.
2. Basics on yabadaba.settings.
3. How to implement more advanced settings modifications.

In [1]:
# Standard Python libraries
import datetime

# Import the main yabadaba package
import yabadaba

# Show yabadaba version
print('yabadaba version =', yabadaba.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

yabadaba version = 0.3.0
Notebook executed on 2025-02-13


## 1. Saving and loading database access and authentication information

There are two things common to database interactions:

1. Data of a given type tends to be hosted in a single location or a small number of user-accessible duplicates.
2. Correctly specifying the access and authentication information for a given database can often be tricky.

Because of these points, yabadaba offers the ability to save the access and authentication details for different databases that can later be easily retrieved using simple names.

### 1.1. View and load a database from the settings

__yabadaba.settings.list_databases__ lists the names for all currently set databases.

In [2]:
yabadaba.settings.list_databases

['dbliquid', 'potentials', 'mongo']

The __yabadaba.load_database()__ method has access to the settings and can load database settings according to a database name. Since 'name' is the first keyword parameter for the method, the database name is all that is needed to load a pre-defined database from settings.

In [3]:
database = yabadaba.load_database('dbliquid')
print(database)

database style local at /home/lmh1/dbliquid_data


### 1.2. Saving new database settings

The access and authentication settings for a database can be saved using __yabadaba.settings.set_database()__.

__Caution!!__ All specified settings will be saved to an unencrypted settings file (see below). As such, be careful with what settings you save if others have access to your home directory.

Basic parameters:

- __name__ (*str*) is the name to assign the settings to.
- __style__ (*str*) is the style/type of database that is being accessed.
- __host__ (*str*) is the host location for the database.  This is usually a URL or a file path.
- __\*\*kwargs__ (*any, optional*) are any other database-style-specific parameters needed to properly access the database.  If no kwargs are specified, there will be prompts asking for them by giving a key then a value. until done.

Parameters specific to the 'local' database style:

- __format__ (*str, optional*) The file format that is used: "json" or "xml".  Default value is "json".
- __indent__ (*int or None, optional*) The number of indentation spacings to use in the files.  If None, then the files will be compact.  Default value is 4.

Parameters specific to the 'cdcs' database style:

- __username__ (*str, optional*) The username to use for accessing the database. An empty str "" indicates that the access is anonymous (no login required).
- __password__ (*str, optional*) The password associated with username to use for accessing the database. If a password is needed and not saved, then a prompt will ask for it whenever a database class is initialized.
- __cert__ (*str, optional*) The path to a certification file if needed for accessing the database.
- __certification__ (*str, optional*) Alias for cert. Retained for compatibility.
- __verify__ (*bool or str, optional*) Either a boolean, in which case it controls whether the server's TLS certificate is verified, or a string, in which case it must be a path to a CA bundle to use. Defaults to True.
- __cdcsversion__ (*str, optional*) For CDCS versions 2.X.X, this allows for specifying the full CDCS version to ensure the correct REST calls are used.  This can be specified as "#.#.#", or if None is given will default to "2.15.0".  For CDCS versions 3.X.X, this is ignored as version info is obtained directly from the database.

Parameters specific to the 'mongo' database style:

- __port__ (*int*) The port to use in connecting to the mongo host.
- __database__ (*str*) The name of the database in the mongo host to interact with.
- __**kwargs__ (*dict, optional*) Any extra keyword arguments needed to initialize a pymongo.MongoClient object.

In [4]:
# Set a local database. Note that a prompt appears as no additional kwargs are given
yabadaba.settings.set_database(name='test', style='local', host='home/lmh1/ipr/test')

Enter any other database parameters as key, value
Exit by leaving key blank
key: 

In [5]:
# Set a CDCS database with anonymous access
yabadaba.settings.set_database(name='potentials', style='cdcs', host='https://potentials.nist.gov/', username='')

Database potentials already defined.
Overwrite? (yes or no): 

 no


In [6]:
# Set a local-hosted MongoDB
yabadaba.settings.set_database(name='mongo', style='mongo', host='localhost', port=27017, database='iprPy')

Database mongo already defined.
Overwrite? (yes or no): 

 no


### 1.3. Unsetting database settings

Any saved database settings can be deleted using the unset_database() method and giving the name of the database settings to delete.

In [8]:
yabadaba.settings.unset_database(name='test')

Database test found
Delete settings? (must type yes): 

 yes


## 2. Basics on yabadaba settings

Settings are managed in yabadaba in the following way:

- The yabadaba.Settings.Settings (uppercase s's) class defines the methods that allow for saving and loading settings.
- yabadaba.settings (lowercase s) is an object of the above class that is created when yabadaba is imported.
- When settings are changed, the updates are made in a settings.json file that can be later loaded and used. The default location for this file is in a directory called ".NIST" located in the user's home directory.

View the current paths with __yabadaba.settings.directory__ and __yabadaba.settings.filename__.

In [9]:
print(yabadaba.settings.directory)
print(yabadaba.settings.filename)

/home/lmh1/.NIST
/home/lmh1/.NIST/settings.json


If needed, the settings directory can be changed to a different location using __yabadaba.settings.set_directory()__.  This likely is not a typical 

Note that set_directory() only works if the settings in the default directory are "empty". Also, the settings.json in the default directory will still exist and serve as a pointer to the new directory that has been set.

In [10]:
#yabadaba.settings.set_directory('/home/lmh1/.yabadaba')
print(yabadaba.settings.directory)

/home/lmh1/.NIST


The settings directory can be reverted back to the default by using __yabadaba.settings.unset_directory()__.

In [11]:
yabadaba.settings.unset_directory()
print(yabadaba.settings.directory)

Settings directory is already set to "/home/lmh1/.NIST"
/home/lmh1/.NIST


## 3. How to implement more advanced settings modifications.

For developers of yabadaba-based packages, you may wish to modify the settings capability.  This can be managed by having your package create its own settings object.

### 3.1. Package-specific settings

When initializing a Settings object, you can specify the default directory name and settings filename to be something different. This can be used to avoid settings conflicts between different packages, *but* prevents different yabadaba-based packages from accessing the same saved database settings.

In [12]:
newsettings = yabadaba.Settings.Settings(directoryname='.mypackage', filename='mysettings.json')
newsettings.filename

PosixPath('/home/lmh1/.mypackage/mysettings.json')

### 3.2. Update load_database()

If you create your own settings object, then you need to update the load_database() function to recognize the changes.

- yabadaba.load_database() has a settings parameter where a different settings object can be specified.
- In your package, simply define your own load_database() method that changes the default settings value to be your settings object before calling yabadaba.load_database().

### 3.3. Subclassing Settings

The yabadaba.Settings.Settings class can easily be extended to managing additional settings by creating your own subclass of it. In this way, you can have package-specific settings that can be set during one session and loaded during another. Some hints/notes/suggestions if you wish to do this are

- The __content__ attribute is the (embedded) dict representation of the settings file. The __load()__ method will read the settings file and update content, while the __save()__ method will save the current content to the settings file.
- If you add new settings options, it is recommended to use a similar naming convention as the directory and database methods above: "mysetting" or "list_mysettings" as attributes to view current values, "set_mysetting()" and "unset_mysetting()" to save and clear saved setting values.
- The set and unset methods should update the appropriate fields in content, then call save().
- While you can save your package-specific settings anywhere in the settings file content, it is probably a good idea to group them together in a sub-dictionary named after your package.  This will help in avoiding settings conflicts if multiple yabadaba-based packages are imported.

Basic template for adding a new setting term
```python
class Settings(yabadaba.Settings.Settings):

    @property
    def mysetting(self):
        """The current value of mysetting"""
        defaultvalue = None

        if 'mypackage' in self.content and 'mysetting' in self.content['mypackage']:
            return self.content['mypackage']['mysetting']
        else:
            return defaultvalue

    def set_mysetting(self, value):
        """Set the value of mysetting"""
        # Do some code to check value's data type and value...

        if 'mypackage' not in content:
            content['mypackage'] = {}

        # Optional check if value is already set
        #if 'mysetting' in self.content['mypackage']
        #    print('mysetting already set')
        #    prompt if you want to change...
        
        content['mypackage']['mysetting'] = value
            
        self.save()

    def unset_mysetting(self):
        """Reset mysetting back to it's default value"""
        if 'mypackage' in self.content and 'mysetting' in self.content['mypackage']:
            del self.content['mypackage']['mysetting']  
        #else:
        #    print('no setting found for mysetting')
        
        self.save()
```